# CV RAG Query Notebook (Deep Dive - Hybrid Search)

V tomto notebooku rozebereme dotazování na prvočinitele.
Použijeme speciální metody v kódu, které nám "vyzradí", jak se Hybrid Search rozhodoval.

Co uvidíme:
1.  Co našla BM25 část (Top K).
2.  Co našla Vektorová část (Top K).
3.  **SIMULACE:** Ručně spočítáme RRF skóre přímo v notebooku.
4.  **METRIKY:** Změříme shodu mezi oběma metodami.

In [1]:
import logging
from pathlib import Path
import sys
import pandas as pd

sys.path.insert(0, str(Path.cwd().parent))

logging.basicConfig(level=logging.ERROR) # Méně logů, více vlastních výpisů

from src.config import AppConfig
from src.embeddings import EmbeddingsManager
from src.vector_store import VectorStoreManager
from src.parent_retriever import CVParentRetriever

In [5]:
# Setup
config = AppConfig()
config.rag.use_hybrid_search = True
embeddings_mgr = EmbeddingsManager(config.azure)
vs_manager = VectorStoreManager(config.rag, embeddings_mgr.get_embeddings())
vectorstore = vs_manager.load_vectorstore()
retriever = CVParentRetriever(config.rag, vectorstore, config.azure)
retriever.load_from_existing_store()

print("✅ Retriever připraven.")


C:\Users\jan.petr\OneDrive - dolphinconsulting.cz\Projects\rag-training\app_cvs\src\vector_store.py:91: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the `langchain-chroma package and should be used instead. To use it run `pip install -U `langchain-chroma` and import as `from `langchain_chroma import Chroma``.
  self._vectorstore = Chroma(


## ČÁST 1: Raw Data (Co vrací BM25 a Vektory?)
Nejdříve si vytáhneme surové výsledky.

In [4]:
query = "python developer"

print(f"🔍 Dotaz: '{query}'")

# Vytáhneme data přímo z interních retrieverů
hybrid_core = retriever._hybrid_retriever

# 1. BM25
bm25_results = hybrid_core.bm25_retriever.invoke(query)
# Ořízneme pro přehlednost na Top 5
bm25_results = bm25_results[:5]

# 2. Vektory
embedding_results = hybrid_core.embedding_retriever.invoke(query)
# Ořízneme pro přehlednost na Top 5
embedding_results = embedding_results[:5]

print(f"Získáno {len(bm25_results)} BM25 výsledků a {len(embedding_results)} Vektorových výsledků.")

🔍 Dotaz: 'python developer'
Získáno 5 BM25 výsledků a 5 Vektorových výsledků.


## ČÁST 2: SIMULACE RRF (Reciprocal Rank Fusion)
Zde vidíte PŘESNĚ ten kód, který se děje uvnitř. Spojujeme výsledky na základě pořadí (ranku).

Vzorec pro skóre dokumentu `d`:
$$ Score(d) = \sum \frac{1}{k + rank(d)} $$

Kde `k` je konstanta (zde 60) a `rank` je pořadí (0, 1, 2...).

In [10]:
for rank, doc in enumerate(bm25_results):
    print(rank)

0
1
2
3
4


In [7]:
from collections import defaultdict

# Konstanty
K = 60
BM25_WEIGHT = 1.0
VECTOR_WEIGHT = 1.0

rrf_scores = defaultdict(float)
# Mapa pro uložení objektů dokumentů, abychom je nekopírovali
doc_map = {}

print("--- ZAČÁTEK FÚZE ---")

# 1. Zpracování BM25
print("\n1. Počítám skóre z BM25:")
for rank, doc in enumerate(bm25_results):
    # Vytvoříme unikátní klíč (Jméno + kousek textu), protože ID se může lišit
    doc_key = f"{doc.metadata.get('candidate_name')}_{doc.page_content[:20]}"
    
    score = BM25_WEIGHT * (1 / (K + rank + 1))
    rrf_scores[doc_key] += score
    doc_map[doc_key] = doc
    
    print(f"   Rank {rank+1}: {doc_key:<30} -> +{score:.5f}")

# 2. Zpracování Vektorů
print("\n2. Přičítám skóre z Vektorů:")
for rank, doc in enumerate(embedding_results):
    doc_key = f"{doc.metadata.get('candidate_name')}_{doc.page_content[:20]}"
    
    score = VECTOR_WEIGHT * (1 / (K + rank + 1))
    
    # Zde vidíme, zda už klíč existuje (Overlap)
    if doc_key in rrf_scores:
        print(f"   Rank {rank+1}: {doc_key:<30} -> +{score:.5f} (MATCH! Zvyšuji skóre)")
    else:
        print(f"   Rank {rank+1}: {doc_key:<30} -> +{score:.5f} (Nový)")
        
    rrf_scores[doc_key] += score
    # Pokud je to nový dokument, uložíme si ho
    if doc_key not in doc_map:
        doc_map[doc_key] = doc

# 3. Seřazení
sorted_docs = sorted(rrf_scores.items(), key=lambda x: x[1], reverse=True)

print("\n🏆 VÝSLEDNÉ POŘADÍ (Top 5):")
for i, (key, score) in enumerate(sorted_docs[:5], 1):
    print(f"   {i}. Skóre {score:.5f} | {key}")

--- ZAČÁTEK FÚZE ---

1. Počítám skóre z BM25:
   Rank 1: Látal Michael_www.dolphinconsultin -> +0.01639
   Rank 2: Huňa Tomáš_www.dolphinconsultin -> +0.01613
   Rank 3: Bobůrka Vojtěch_www.dolphinconsultin -> +0.01587
   Rank 4: Konvalinka Michal_www.dolphinconsultin -> +0.01562
   Rank 5: Hušek Michal_www.dolphinconsultin -> +0.01538

2. Přičítám skóre z Vektorů:
   Rank 1: Bukovský Petr_Javascript (scriptin -> +0.01639 (Nový)
   Rank 2: Baláček Daniel_PowerDesigner

Appli -> +0.01613 (Nový)
   Rank 3: Hušek Michal_www.dolphinconsultin -> +0.01587 (MATCH! Zvyšuji skóre)
   Rank 4: Hlinková Zuzana_www.dolphinconsultin -> +0.01562 (Nový)
   Rank 5: Huňa Tomáš_Cloud environment

A -> +0.01538 (Nový)

🏆 VÝSLEDNÉ POŘADÍ (Top 5):
   1. Skóre 0.03126 | Hušek Michal_www.dolphinconsultin
   2. Skóre 0.01639 | Látal Michael_www.dolphinconsultin
   3. Skóre 0.01639 | Bukovský Petr_Javascript (scriptin
   4. Skóre 0.01613 | Huňa Tomáš_www.dolphinconsultin
   5. Skóre 0.01613 | Baláček Daniel_Po

## ČÁST 3: METRIKY (Shoda)
Jak moc se výsledky překrývaly? Pokud je překryv malý, znamená to, že každá metoda našla úplně něco jiného (což je u Hybrid Search vlastně dobře - doplňují se).

Použijeme **Jaccardovu podobnost**:
$$ J(A,B) = \frac{|A \cap B|}{|A \cup B|} $$

In [11]:
# Vytvoříme množiny klíčů
bm25_keys = set()
for doc in bm25_results:
    key = f"{doc.metadata.get('candidate_name')}_{doc.page_content[:20]}"
    bm25_keys.add(key)

vector_keys = set()
for doc in embedding_results:
    # U vektorů musíme být opatrní - vrací child chunks. 
    # Pro srovnání bychom měli porovnávat spíše JMÉNA kandidátů, 
    # protože text bude vždy jiný (odstavec vs celé CV).
    key = f"{doc.metadata.get('candidate_name')}_{doc.page_content[:20]}"
    vector_keys.add(key)

# Průnik a Sjednocení
intersection = bm25_keys.intersection(vector_keys)
union = bm25_keys.union(vector_keys)

jaccard_index = len(intersection) / len(union) if len(union) > 0 else 0

print(f"\n📊 METRIKY SHODY:")
print(f"   Počet BM25: {len(bm25_keys)}")
print(f"   Počet Vector: {len(vector_keys)}")
print(f"   Průnik (Identické chunk texty): {len(intersection)}")
print(f"   Jaccardova podobnost: {jaccard_index:.2%}")

if jaccard_index == 0:
    print("\n💡 INTERPRETACE: Jaccard 0% je zde OČEKÁVANÝ.")
    print("   BM25 vrací 'Parent Chunks' (dlouhé texty).")
    print("   Vector Store vrací 'Child Chunks' (krátké úryvky).")
    print("   Jejich začátek (page_content[:20]) se může, ale nemusí shodovat.")


📊 METRIKY SHODY:
   Počet BM25: 5
   Počet Vector: 5
   Průnik (Identické chunk texty): 1
   Jaccardova podobnost: 11.11%


In [13]:
print(bm25_keys)
print(vector_keys)
print(intersection)
print(union)

{'Hušek Michal_www.dolphinconsultin', 'Látal Michael_www.dolphinconsultin', 'Huňa Tomáš_www.dolphinconsultin', 'Konvalinka Michal_www.dolphinconsultin', 'Bobůrka Vojtěch_www.dolphinconsultin'}
{'Baláček Daniel_PowerDesigner\n\nAppli', 'Hušek Michal_www.dolphinconsultin', 'Huňa Tomáš_Cloud environment\n\nA', 'Hlinková Zuzana_www.dolphinconsultin', 'Bukovský Petr_Javascript (scriptin'}
{'Hušek Michal_www.dolphinconsultin'}
{'Hušek Michal_www.dolphinconsultin', 'Huňa Tomáš_Cloud environment\n\nA', 'Hlinková Zuzana_www.dolphinconsultin', 'Bukovský Petr_Javascript (scriptin', 'Látal Michael_www.dolphinconsultin', 'Huňa Tomáš_www.dolphinconsultin', 'Konvalinka Michal_www.dolphinconsultin', 'Bobůrka Vojtěch_www.dolphinconsultin', 'Baláček Daniel_PowerDesigner\n\nAppli'}


### Alternativní metrika: Shoda na úrovni Kandidátů
Protože RAG v posledním kroku (Parent Retriever) dohledává rodičovský dokument, dává větší smysl měřit, zda obě metody našly **stejné lidi** (kandidáty), i když přes jiné kousky textu.

In [14]:
bm25_names = {doc.metadata.get('candidate_name') for doc in bm25_results}
vector_names = {doc.metadata.get('candidate_name') for doc in embedding_results}

intersection_names = bm25_names.intersection(vector_names)
union_names = bm25_names.union(vector_names)

jaccard_names = len(intersection_names) / len(union_names) if len(union_names) > 0 else 0

print(f"\n📊 SHODA KANDIDÁTŮ (Jména):")
print(f"   BM25 našel: {bm25_names}")
print(f"   Vector našel: {vector_names}")
print(f"   Společní kandidáti: {intersection_names}")
print(f"   Jaccardova podobnost (Kandidáti): {jaccard_names:.2%}")

if jaccard_names > 0:
    print("\n✅ Vidíte! I když texty (chunks) byly jiné, obě metody často najdou stejné relevantní lidi.")
else:
    print("\n❌ Každá metoda našla úplně jiné lidi. To se stává u specifických dotazů.")


📊 SHODA KANDIDÁTŮ (Jména):
   BM25 našel: {'Konvalinka Michal', 'Bobůrka Vojtěch', 'Látal Michael', 'Huňa Tomáš', 'Hušek Michal'}
   Vector našel: {'Bukovský Petr', 'Baláček Daniel', 'Hlinková Zuzana', 'Huňa Tomáš', 'Hušek Michal'}
   Společní kandidáti: {'Huňa Tomáš', 'Hušek Michal'}
   Jaccardova podobnost (Kandidáti): 25.00%

✅ Vidíte! I když texty (chunks) byly jiné, obě metody často najdou stejné relevantní lidi.
